In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import matplotlib.pyplot as plt
navigator = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1)'

In [2]:
def info_films(id):
  url_base = 'https://www.imdb.com'
  url_base_title = 'https://www.imdb.com/fr/title/'
  url_finale_title = f'{url_base_title}{id}'

  #TRAILER

  html_title = requests.get(url_finale_title, headers={'User-Agent': navigator})
  html_title2 = html_title.content
  soup_title = BeautifulSoup(html_title2, 'html.parser')

  for balise_parent in soup_title.find_all('div', class_='ipc-page-content-container ipc-page-content-container--center'):
    for element in balise_parent.find_all('a', class_='ipc-lockup-overlay ipc-focusable'):
      try:
        if 'video' in element['href']:
          trailer = element['href']
          break
      except:
        pass

  lien_trailer = f'{url_base}{trailer}'

  #AFFICHE

  html_affiche = requests.get(url_finale_title, headers={'User-Agent': navigator})
  html_affiche2 = html_affiche.content
  soup_affiche = BeautifulSoup(html_affiche2, 'html.parser')
  affiche = ''

  for balise_parent in soup_affiche.find_all('div', class_='ipc-page-content-container ipc-page-content-container--center'):
    for element in balise_parent.find_all('img', class_='ipc-image'):
      affiche += f", {element['src']}"

  affiche = affiche.split(', ')

  if "" in affiche:
    affiche.remove("")

  lien_affiche = affiche[0]

  #ACTEURS

  html_acteurs = requests.get(url_finale_title, headers={'User-Agent': navigator})
  html_acteurs2 = html_acteurs.content
  soup_acteurs = BeautifulSoup(html_acteurs2, 'html.parser')
  liste_acteurs = []
  for balise_parent in soup_acteurs.find_all('div', class_='sc-cd7dc4b7-7 vCane'):
    for element in balise_parent.find_all('a', class_='sc-cd7dc4b7-1 kVdWAO'):
      liste_acteurs.append(element.get_text().strip())

  if len(liste_acteurs) > 4:
    liste_acteurs = liste_acteurs[:4]

  #PHOTOS ACTEURS

  html_acteurs = requests.get(url_finale_title, headers={'User-Agent': navigator})
  html_acteurs2 = html_acteurs.content
  soup_acteurs = BeautifulSoup(html_acteurs2, 'html.parser')
  dico_photos = {}
  dico_photos_final = {}

  for balise_parent in soup_acteurs.find_all('img', class_='ipc-image'):
      dico_photos.update({balise_parent['alt'] : balise_parent['src']})

  for element in dico_photos.keys():
      if element in liste_acteurs:
          dico_photos_final.update({element : dico_photos[element]})

    #REALISATEUR

  html_realisateurs = requests.get(url_finale_title, headers={'User-Agent': navigator})
  html_realisateurs2 = html_realisateurs.content
  soup_realisateurs = BeautifulSoup(html_realisateurs2, 'html.parser')
  liste_realisateurs = []
  for balise_parent in soup_affiche.find_all('a', class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link'):
    liste_realisateurs.append(balise_parent.get_text().strip())

  realisateur = liste_realisateurs[0]

  #RESUME

  html_resume = requests.get(url_finale_title, headers={'User-Agent': navigator})
  html_resume2 = html_resume.content
  soup_resume = BeautifulSoup(html_resume2, 'html.parser')

  for balise_parent in soup_affiche.find_all('span', class_='sc-3ac15c8d-0 hRUoSB'):
    resume = balise_parent.get_text().strip()

  if '... Tout lire' in resume:
    resume = resume.replace('Tout lire', '')

  return lien_trailer, lien_affiche, liste_acteurs, dico_photos_final, realisateur, resume

In [3]:
trailer, affiche, acteur, photos, realisateur, resume = info_films('tt0473829')

In [4]:
trailer

'https://www.imdb.com/video/vi3256401433/?ref_=tt_vi_i_1'

In [5]:
affiche

'https://m.media-amazon.com/images/M/MV5BOTNkOGZjMTQtYzEzZS00OGJkLTk0OGItMzRhMTkzOGIyYmMxXkEyXkFqcGc@._V1_QL75_UY281_CR5,0,190,281_.jpg'

In [6]:
realisateur

'Fabien Onteniente'

In [7]:
acteur

['Gérard Lanvin', 'Mathilde Seigner', 'Franck Dubosc', 'Claude Brasseur']

In [8]:
photos

{'Gérard Lanvin': 'https://m.media-amazon.com/images/M/MV5BMjEwNTA0MDM2MV5BMl5BanBnXkFtZTcwMzkyNTgyMg@@._V1_QL75_UY140_CR1,0,140,140_.jpg',
 'Mathilde Seigner': 'https://m.media-amazon.com/images/M/MV5BMTY4OTQxMzg0MF5BMl5BanBnXkFtZTcwMDg0NzU0Nw@@._V1_QL75_UX140_CR0,12,140,140_.jpg',
 'Franck Dubosc': 'https://m.media-amazon.com/images/M/MV5BOGEyMzgzMDItM2Q5Yy00ZWJhLWFkZDQtODEyMWYyMDM1NjBhXkEyXkFqcGc@._V1_QL75_UX140_CR0,12,140,140_.jpg',
 'Claude Brasseur': 'https://m.media-amazon.com/images/M/MV5BZjg1ZjEyNTYtM2ZmMS00ZGE5LWI3ZTgtMTA1Mzc5YzBjN2YyXkEyXkFqcGc@._V1_QL75_UX140_CR0,0,140,140_.jpg'}

In [9]:
resume

"Au camping des Flots Bleus, ça débarque de toute la France. Comme tous les ans, c'est le moment des retrouvailles autour de l'apéro d'usage pour les familles d'habitués. Sauf que cette année... "